In [1]:
# Para instalar o selenium digite no cmd: pip install selenium
# Necessário também baixar o webdriver de acordo com o seu navegador, o chromedriver já está na página do projeto, pode usar ele
# Uso o Chrome, baixei um arquivo no site e colei na pasta do projeto.
from selenium import webdriver # biblioteca que vai abrir no navegador, apertar os botões e dar os htmls
#from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By # Usado para identificar as coisas no site(botões, campos de texto)
from time import sleep # sleep(x) faz o programa aguardar x segundos para prosseguir para a próxima linha de código.
from bs4 import BeautifulSoup # Pega um código html e nos permite usar funções especiais para explorá-lo
import pandas as pd # Usado para criar e manipular Dataframes(objetos que representam planilhas)
import openpyxl # Vai servir como parâmentro na função ExcelWriter, para podermos manipular os arquivos excel(xlsx)

In [2]:
lista_links = [10323, 10325, 10327, 10343, 10345, 10347]
"""
Os links dos resultados seguem o seguinte formato:
'http://qselecao.ifce.edu.br/resultado_nomes.aspx?COD_REGRA_CLASSIFICACAO=XXXXX'
O que muda é apenas o XXXXX, sendo eles um dos números da lista_links, cada link é a tabela de resultados de um curso.
Resolvi criar essa lista para não ter que escrever o http://qselecao... 6 vezes.
"""

In [10]:
# CRIEI UM ARQUIVO EXCEL, MANUALMENTE, CHAMADO resultado PARA ACESSÁ-LO NO CÓDIGO E INSERIR OS DATAFRAMES
# Abre o navegador no modo automático
navegador = webdriver.Chrome() 

listaDfPrincipal = [] # Cada item dessa lista será um dataframe de um curso

posDfPrincipal = 0 # Posição inicial na lista
for i in lista_links: # Para cada número na lista_links, ou seja, para cada curso
    
    # Vai no navegador que foi aberto no modo automático e acessará o link
    navegador.get('http://qselecao.ifce.edu.br/resultado_nomes.aspx?COD_REGRA_CLASSIFICACAO='+str(i))
    
    sleep(2) # Aguarda 3 segundos
    
    # Vai procurar o elemento com o id que foi passado. Esse é o id do botão classificação. Eu fui no site e inspecionei 
    # o html para encontrar esse id.
    # O parâmetro By.ID vem da biblioteca By, serve para procurar elementos pela id da tag
    botaoClassificacao = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolderPrincipal_rblOrdenacao_1')
    
    # Vai clicar no botão que foi encontrado na variável anterior
    botaoClassificacao.click() 
    
    sleep(1)
    
    # Vai usar o construtor BeautifulSoup, passando a html do navegador e armazenando na variável soup
    # Assim podemos usar os métodos que a biblioteca disponibiliza
    soup = BeautifulSoup(navegador.page_source, 'html.parser')
    
    # A variável tabela vai armazenar o elemento do html que contém o id especificado
    # O método find faz essa busca
    # Aqui eu não usei o BY.id por que um objeto BeautifulSoup já tem esse parâmetro por padrão, basta indicar o id
    tabela = soup.find(id='ctl00_ContentPlaceHolderPrincipal_wucResultados1_grvConsulta')
    
    # Aqui usei o Pandas para ler o conteúdo da variável tabela, e armazenar na variável listaDataframes.
    listaDataframes = pd.read_html(str(tabela)) # Essa função retorna uma lista com o dataframe dentro dela = [dataframe]
    
    # A lista principal vai colocar o df que foi obtido anteriormente na última posição
    listaDfPrincipal.append(listaDataframes[0])
    
    sleep(1)
    
    for a in range(8):# Para cada número de 0 a 7(total de 8 números)
        
        # O id dos botões de cotas segue o seguinte formato:
        # ctl00_ContentPlaceHolderPrincipal_wucSelecionarCota1_rblCotas_X, sendo esse x no final um número de 1 a 8
        # que representa as cotas de L1 a l8
        # Somei a com 1 pois os valores de a vão de 0 a 7, aqui queremos de 1 a 8
        # O parâmetro By.ID vem da biblioteca By, serve para procurar elementos pela id da tag
        botaoCotas = navegador.find_element(By.ID, 'ctl00_ContentPlaceHolderPrincipal_wucSelecionarCota1_rblCotas_'+str(a+1))
        
        # Clica no botão cotas
        botaoCotas.click() 
        
        # A variável page vai receber o html completo da página
        page = navegador.page_source
        
        # A variável soup recebe a função BeautifulSoup, que pega o html de page e torna possível usar os métodos da BS nele
        soup = BeautifulSoup(page, 'html.parser')
        
        # A variável tabelaCotas vai pegar a soup criada anteriormente e usar o método find nela
        # para pegar o html da tabela, que contém o id especificado.
        # Aqui eu não usei o BY.id por que um objeto BeautifulSoup já tem esse parâmetro por padrão, basta indicar o id
        tabelaCotas = soup.find(id='ctl00_ContentPlaceHolderPrincipal_wucResultados1_grvConsulta')
        
        # Verifica se foi encontrada alguma tabela na cota atual, caso sim, executa o código seguinte
        if tabelaCotas != None:
            
            # Transforma a tabela em um dataframe. A saída vai ser uma lista com o dataframe da tabela dentro. [dataframe]
            dfTabelaCotas = pd.read_html(str(tabelaCotas))
            
            # O dataframe atual que está dentro da listaDfPrincipal vai receber ele mesmo mais a tabela de cota que foi 
            # encontra. Ou seja, vai pegar a tabela da cota e encaixar embaixo da lista que foi encontrada antes.
            # Como dfTabelaCotas é uma lista com o dataframe dentro, e eu queria só o dataframe, coloquei: [0]
            listaDfPrincipal[posDfPrincipal] = pd.concat([listaDfPrincipal[posDfPrincipal], dfTabelaCotas[0]])
       
    sleep(1)
    posDfPrincipal += 1 # Avança na lista principal para podermos inserir um novo dataframe de outro curso


0
0
0
0
1
1
1
2
2
2
2
2
3
3
3
4
4


In [11]:
for i in listaDfPrincipal:
    display(i)

,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,42,Miguel Luis Costa Aguiar,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,847,1,Aprovado
1,8,Arthur Bruno Nunes Barbosa,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,830,2,Aprovado
2,2,Alana Késsia Bandeira Cavalcante,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,766,3,Aprovado
3,19,Iasmim Braga Costa,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,744,4,Aprovado
4,1,Alan Mateus de Almeida Queiroz,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,704,5,Aprovado
5,20,Iasmin Lopes Barreto,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,676,6,Aprovado
6,9,Carlos Gabriel Clementino Sores,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,675,7,Aprovado
7,17,Guilherme Caetano dos Santos,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,673,8,Aprovado
8,33,Marcley Barbosa da Silva Filho,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,657,9,Aprovado
9,21,Isaac Maciel Lima de Abreu,Técnico Integrado em Edificações - Campus Fort...,Ampla Concorrência,630,10,Aprovado


,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,57,Beatriz Nobre Leite,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,767,1,Aprovado
1,60,Davi Ribeiro da Silva,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,727,2,Aprovado
2,65,Giovanna Sousa Gomes,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,723,3,Aprovado
3,55,Arthur Julião Correia,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,700,4,Aprovado
4,82,Pedro Lucas Rodrigues de Brito Amaro,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,693,5,Aprovado
5,84,Sara Jamilly Moreira da Silva,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,681,6,Aprovado
6,56,Beatriz Monte Queiroz,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,675,7,Aprovado
7,80,Pablo Paulino Frores,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,643,8,Aprovado
8,70,José Henrique de Oliveira Pereira,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,622,9,Aprovado
9,77,Luiz Gabriel de Oliveira Souza,Técnico Integrado em Eletrotécnica - Campus Fo...,Ampla Concorrência,615,10,Aprovado


,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,99,Anna Júlia Rodrigues Andrade Nunes,Técnico Integrado em Informática - Campus Fort...,Ampla Concorrência,884,1,Aprovado
1,266,Vitor Adriano Carneiro Lira,Técnico Integrado em Informática - Campus Fort...,Ampla Concorrência,852,2,Aprovado
2,267,Vítor Araújo Coelho,Técnico Integrado em Informática - Campus Fort...,Ampla Concorrência,815,3,Aprovado
3,217,Matheus Pereira Menezes,Técnico Integrado em Informática - Campus Fort...,Ampla Concorrência,800,4,Aprovado
4,223,Miguel da Silva Souza Ribeiro,Técnico Integrado em Informática - Campus Fort...,Ampla Concorrência,784,5,Aprovado
...,...,...,...,...,...,...,...
6,146,Graziele Lima da Silva,Técnico Integrado em Informática - Campus Fort...,"L6 - Candidatos autodeclarados pretos, pardos ...",501,7,Classificável
7,162,Ívila Cris de Sousa Rocha,Técnico Integrado em Informática - Campus Fort...,"L6 - Candidatos autodeclarados pretos, pardos ...",361,8,Classificável
0,92,Ana Livia Oliveira Lima,Técnico Integrado em Informática - Campus Fort...,L8 - Candidatos que independentemente da renda...,570,1,Aprovado
1,148,Gustavo de Oliveira Félix Santos,Técnico Integrado em Informática - Campus Fort...,L8 - Candidatos que independentemente da renda...,460,2,Classificável


,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,295,Leonardo Delon Sampaio Araujo,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,840,1,Aprovado
1,303,Ryan Lucas de Souza Oliveira,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,804,2,Aprovado
2,279,Clara Sousa Bandeira,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,776,3,Aprovado
3,292,João Vitor Santos Silva,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,764,4,Aprovado
4,305,Yago Pontes Freitas,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,729,5,Aprovado
5,277,Angelina Thaila Sousa dos Santos,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,719,6,Aprovado
6,276,Ana Beatriz de Oliveira Chagas,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,698,7,Aprovado
7,287,Guilherme Batista Chavante,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,693,8,Aprovado
8,296,Lucas Vladson Correia de Lima,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,692,9,Aprovado
9,291,Isacc Santos Faustino,Técnico Integrado em Mecânica Industrial - Cam...,Ampla Concorrência,691,10,Aprovado


,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,369,Wladson Rodrigues de Queiroz,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,819,1,Aprovado
1,321,Ian Barreto Tavares Matos de Vasconcelos,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,807,2,Aprovado
2,306,Adrielly Maria Milhome Pereira,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,801,3,Aprovado
3,355,Pedro Héctor de Sousa Carneiro,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,799,4,Aprovado
4,341,Maria Eduarda Magalhães de Oliveira,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,793,5,Aprovado
5,332,Laura de Souza Marques,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,781,6,Aprovado
6,350,Marília Colaço Pinheiro Bezerra,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,754,7,Aprovado
7,316,Erick Luan Mota Albuquerque,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,747,8,Aprovado
8,335,Lohana Mesquita Monteiro Lobo,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,740,9,Aprovado
9,361,Stéfany Araújo de Sousa,Técnico Integrado em Química - Campus Fortaleza,Ampla Concorrência,733,10,Aprovado


,Insc.,Nome,Curso,Reserva de Vaga,Pontuação,Classificação,Situação
0,377,Denyse Pinheiro de Zamenhof,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,827,1,Aprovado
1,375,Dayane Pinheiro de Zamenhof,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,812,2,Aprovado
2,400,Tiago Gomes Oliveira,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,791,3,Aprovado
3,371,Ana Rebeca Bastos de Azevedo,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,790,4,Aprovado
4,390,Maria Flor Carvalho Guerra,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,776,5,Aprovado
5,376,Deborah da Silva Magalhães,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,761,6,Aprovado
6,372,Ana Valentina Torres Memoria,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,750,7,Aprovado
7,404,Yasmin Evelyn Dias Moreira,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,743,8,Aprovado
8,398,Rachel Sucupira Gregorio de Castro,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,729,9,Aprovado
9,392,Mariana Siqueira Holanda Araújo,Técnico Integrado em Telecomunicações - Campus...,Ampla Concorrência,728,10,Aprovado


In [9]:
# O código abaixo limpa a lista de dataframes principal, caso dê algum erro no meio execução, limpe essa lista antes de rodar o 
# código novamente
#listaDataframes.clear()

In [14]:
# Vai acessar o arquivo criado manualmente, mode=a significa que iremos inserir na maneira append(na última posição)
# sheet são as abas(planilhas) que acessamos na parte de baixo do excel
# if_sheet_exists='overlay' significa que caso a planilha com o mesmo nome já exista, vai ser substituida pela que vamos pôr
# Isso não deve acontecer, pois o arquivo estará vazio na hora da criação
# CRIE UM NOVO ARQUIVO EXCEL DENTRO DA PASTA E MUDE O NOME DO resultado ABAIXO PARA TESTAR O CÓDIGO
with pd.ExcelWriter("resultado.xlsx", mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:
    
    # Para cada dataframe na listaDfPrincipal
    for df in listaDfPrincipal:
        # Irá colocá-lo no arquivo resultado.xlsx, que é passado pelo parâmetro writer, criado anteriormente
        # sheet_name é o nome da aba(planilha), é uma para cada curso
        # nesse sheet_name fiz uma adaptação meia boca
        # peguei a coluna Curso do dataframe atual, na linha 9(escolhida aleatoriamente)
        # caracteres do 20 ao 35 da string, faz aparecer o nome do curso, de uma forma estranha, mas faz
        # é necessário renomear os nomes dos cursos depois disso ainda
        df.to_excel(writer, sheet_name=df.Curso[9][20:35])